In [ ]:
# imports
import numpy as np
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, BatchNormalization, Flatten, SimpleRNN, Input, Activation,Add
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D ,ZeroPadding2D
from keras.utils.np_utils import to_categorical
from keras.optimizers import adam_v2, rmsprop_v2
import glob
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split
import cv2
import matplotlib.pyplot as plt
import os

In [ ]:
# global config
map_characters = {0: 'abraham_grampa_simpson', 1: 'apu_nahasapeemapetilon', 2: 'moe_szyslak', 
        3: 'charles_montgomery_burns', 4: 'chief_wiggum', 5: 'comic_book_guy', 6: 'edna_krabappel', 
        7: 'homer_simpson', 8: 'kent_brockman', 9: 'krusty_the_clown', 10: 'lenny_leonard', 11:'lisa_simpson',
        12: 'marge_simpson', 13: 'mayor_quimby',14:'milhouse_van_houten', 15: 'bart_simpson', 
        16: 'ned_flanders', 17: 'nelson_muntz', 18: 'principal_skinner', 19: 'sideshow_bob'}

img_size = 64
batch_size = 32
epochs = 100
num_classes = len(map_characters)
valid_rate = 0.2
initial_lr = 0.001

In [ ]:
# load the test set and preprocess
def load_test_set(dir_name):
    X_test, y_test = [], []
    for image_name in os.listdir(dir_name):
        character_name = '_'.join(image_name.split('_')[:-1])
        label = [label for label, character in map_characters.items() if character == character_name][0]
        image = cv2.imread(dir_name+'/'+image_name)
        image = cv2.resize(image, (img_size, img_size)).astype('float32')/255.
        X_test.append(image)
        y_test.append(label)
    return np.array(X_test), np.array(y_test)

def load_train_set(dir_name):
    X_train, y_train = [], []
    for label, character in map_characters.items():
        list_images = os.listdir(dir_name+'/'+character)
        for image_name in list_images[0:150]:
            image = cv2.imread(dir_name+'/'+character+'/'+image_name)
            image = cv2.resize(image, (img_size, img_size)).astype('float32')/255.
            X_train.append(image)
            y_train.append(label)   
    return np.array(X_train), np.array(y_train)

X_test, y_test = load_test_set("/kaggle/input/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset")
X_train, y_train = load_train_set("/kaggle/input/the-simpsons-characters-dataset/simpsons_dataset/simpsons_dataset")
# transfer labels to one-hot vectors
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)
# split training data to training data and validation data
# valid_num = int(X_train.shape[0] * valid_rate)
# X_train, X_valid = X_train[:-valid_num], X_train[-valid_num:]
# y_train, y_valid = y_train[:-valid_num], y_train[-valid_num:]
(X_train, X_valid, y_train, y_valid) = train_test_split(X_train, y_train, test_size=valid_rate, stratify=y_train)

print(f'X_train shape:{X_train.shape} y_train shape:{y_train.shape}')
print(f'X_valid shape:{X_valid.shape} y_valid shape:{y_valid.shape}')
print(f'X_test shape:{X_test.shape} y_test shape:{y_test.shape}')

In [ ]:
# create cnn model
# VGG-like
def create_VGGLike_model(input_shape=(img_size,img_size,3)):
    model = Sequential()

    model.add(Conv2D(filters=64,kernel_size=(3,3),padding='Same',activation='relu',input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding='Same',activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=128,kernel_size=(3,3),padding='Same',activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128,kernel_size=(3,3),padding='Same',activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=256,kernel_size=(3,3),padding='Same',activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=256,kernel_size=(3,3),padding='Same',activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=256,kernel_size=(3,3),padding='Same',activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters=512,kernel_size=(3,3),padding='Same',activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=512,kernel_size=(3,3),padding='Same',activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=512,kernel_size=(3,3),padding='Same',activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters=512,kernel_size=(3,3),padding='Same',activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=512,kernel_size=(3,3),padding='Same',activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=512,kernel_size=(3,3),padding='Same',activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
#     optimizer = rmsprop_v2.RMSprop(learning_rate=0.0001, decay=1e-6)
    optimizer = adam_v2.Adam(learning_rate=initial_lr, decay=initial_lr / epochs)
    model.compile(loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy'])
#     keras.utils.vis_utils.plot_model(model, to_file='cnn_architecture.png', show_shapes=True, show_layer_names=True)
    print(model.summary())
    return model

model = create_VGGLike_model()

In [ ]:
def identity_block(X, filters):
    f1, f2, f3 = filters
    X_residual = X
    
    X = Conv2D(filters=f1, kernel_size=(1,1))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=f2, kernel_size=(3,3), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=f3, kernel_size=(1,1))(X)
    X = BatchNormalization(axis=3)(X)
    
    X = Add()([X, X_residual])
    X = Activation('relu')(X)
    
    return X
    
def convolution_block(X, filters):
    f1, f2, f3 = filters
    X_residual = X
    
    X = Conv2D(filters=f1, kernel_size=(1,1))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=f2, kernel_size=(3,3), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=f3, kernel_size=(1,1))(X)
    X = BatchNormalization(axis=3)(X)
    
    X_residual = Conv2D(f3, kernel_size=(1,1))(X_residual)
    X_residual = BatchNormalization(axis=3)(X_residual)
    
    X = Add()([X, X_residual])
    X = Activation('relu')(X)
    
    return X

def create_ResNetLike_model(input_shape=(img_size,img_size,3)):
    X_input = Input(input_shape)
    # X = ZeroPadding2D((3,3))(X_input)
    X = Conv2D(64, (7,7), strides=(2,2), padding='same')(X_input)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3,3), strides=(2,2))(X)
    
    X = convolution_block(X, filters=[64,64,256])
    X = identity_block(X, filters=[64,64,256])
    X = identity_block(X, filters=[64,64,256])
    
    X = convolution_block(X, filters=[128,128,512])
    X = identity_block(X, filters=[128,128,512])
    X = identity_block(X, filters=[128,128,512])
    X = identity_block(X, filters=[128,128,512])
    
    X = convolution_block(X, filters=[256,256,1024])
    X = identity_block(X, filters=[256,256,1024])
    X = identity_block(X, filters=[256,256,1024])
    X = identity_block(X, filters=[256,256,1024])
    X = identity_block(X, filters=[256,256,1024])
    X = identity_block(X, filters=[256,256,1024])
    
    X = convolution_block(X, filters=[512,512,2048])
    X = identity_block(X, filters=[512,512,2048])
    X = identity_block(X, filters=[512,512,2048])
    X = AveragePooling2D((2,2))(X)
    
    X = Flatten()(X)
    X = Dense(num_classes, activation='softmax')(X)
    
    model = Model(inputs=X_input, outputs=X)
    model.compile(loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])
    print(model.summary())
    
    return model
    
model = create_ResNetLike_model()

In [ ]:
# data augumentation and fit model
def fit_model(model, X_train, y_train, X_valid, y_valid):
    data_generator = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        vertical_flip=False)
    data_generator.fit(X_train)

    history = model.fit_generator(data_generator.flow(X_train, y_train, batch_size=batch_size),
                                  steps_per_epoch=X_train.shape[0]//batch_size,
                                  validation_data=(X_valid, y_valid),
                                  epochs=epochs)
    
    return model, history

model, history = fit_model(model, X_train, y_train, X_valid, y_valid)

In [ ]:
# create rnn model
time_steps = img_size
input_size = img_size * 3  # reserve rgb
hidden_layer = 256 

X_train = X_train.reshape(-1, time_steps, input_size)
X_test = X_test.reshape(-1, time_steps, input_size)
X_valid = X_valid.reshape(-1, time_steps, input_size)
print(X_train.shape, X_valid.shape)

def create_rnn_model():
    model = Sequential()
    # RNN cell
    model.add(SimpleRNN(hidden_layer))
    model.add(Dense(num_classes, activation='softmax'))
    
    optimizer = rmsprop_v2.RMSprop(learning_rate=0.0001, decay=1e-6)
    model.compile(loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy'])
    
    model.build((None, time_steps, input_size))
    print(model.summary())
    return model

model = create_rnn_model()
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=epochs)

In [ ]:
def plot_training_history(history):
    print(history.history)
    # plot accuracy history
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label='val_accuracy')
    plt.title('Model accuracy')
    plt.ylabel('Accuarcy')
    plt.xlabel('Epoch')
    plt.legend(loc='upper left')
    plt.show()

    # plot loss history
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='upper left')
    plt.show()
    
plot_training_history(history)

In [ ]:
# evaluate model and plot results
def evaluate_model(model, X_test, y_test):
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f'test loss: {test_loss} test accuracy: {test_acc}')
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)
    print(metrics.classification_report(y_true_classes, y_pred_classes, target_names=list(map_characters.values())), sep='')

    plt.figure(figsize=(8,8))
    cfs_matrix = metrics.confusion_matrix(y_true_classes, y_pred_classes)
    plt.imshow(cfs_matrix, interpolation='nearest')
    plt.colorbar()
    character_names = list(map_characters.values())
    tick_labels = np.arange(len(character_names))
    plt.xticks(tick_labels, character_names, rotation=90)
    plt.yticks(tick_labels, character_names)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
evaluate_model(model, X_test, y_test)

In [ ]:
def random_predict():
    image_set = [k for k in glob.glob('/kaggle/input/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset/*.*')]
    plt.figure(figsize=(10,10))
    plt.subplots_adjust(wspace=0, hspace=0)

    for i in range(9):
        img_file = np.random.choice(image_set)
        character_name = [name for name in map_characters.values() if name in img_file][0].split('_')[0].title()
        image = cv2.imread(img_file)
        show_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (img_size, img_size)).astype('float32')/255.
        y_pred = model.predict(image.reshape(1,img_size,img_size,3))[0]
        text = sorted(['{:s} : {:.2f}%'.format(map_characters[k].split('_')[0].title(), 100*v) for k,v in enumerate(y_pred)], 
           key=lambda x:float(x.split(':')[1].split('%')[0]), reverse=True)[:3]

        # resize before showing
        show_image = cv2.resize(show_image, (352,352))
        cv2.rectangle(show_image, (0,260), (215,352),(255,255,255),-1)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(show_image, 'Name: %s'%character_name, (10,280), font, 0.7, (0,0,0), 2, cv2.LINE_AA)
        for k, v in enumerate(text):
            cv2.putText(show_image, v, (10, 300+k*18), font, 0.65, (0,0,0), 2, cv2.LINE_AA)
        plt.subplot(3,3,i+1)
        plt.imshow(show_image);plt.axis('off')

random_predict()